In [1]:
import akshare as ak
import pandas as pd
import sys
sys.path.append('/Users/afirez/studio/python/auto-trade/examples/py_quant/common')
sys.path.append('/Users/afirez/studio/python/auto-trade/examples/py_quant/common/common_pg.py')

print("sys.path:",sys.path)
import common_pg as common_db

print(ak.__version__)

sys.path: ['/Users/afirez/studio/python/auto-trade/examples/py_quant/datasource/akshare/test', '/usr/local/anaconda3/envs/py38/lib/python38.zip', '/usr/local/anaconda3/envs/py38/lib/python3.8', '/usr/local/anaconda3/envs/py38/lib/python3.8/lib-dynload', '', '/usr/local/anaconda3/envs/py38/lib/python3.8/site-packages', '/usr/local/anaconda3/envs/py38/lib/python3.8/site-packages/IPython/extensions', '/Users/afirez/.ipython', '/Users/afirez/.local/lib/python3.8/site-packages', '/Users/afirez/.local/lib/python3.8/site-packages', '/Users/afirez/studio/python/auto-trade/examples/py_quant/common', '/Users/afirez/studio/python/auto-trade/examples/py_quant/common/common_pg.py']
pg_host : 192.168.31.110 ,pg_user : postgres ,pg_db : k_house
1.6.64


In [2]:
# 实时行情数据
# 接口: stock_zh_a_spot
# 目标地址: http://vip.stock.finance.sina.com.cn/mkt/#hs_a
# 描述: A 股数据是从新浪财经获取的数据, 重复运行本函数会被新浪暂时封 IP, 建议增加时间间隔
# 限量: 单次返回所有 A 股上市公司的实时行情数据

stock_zh_a_spot_df = ak.stock_zh_a_spot()
print(stock_zh_a_spot_df)

            代码    名称     最新价    涨跌额     涨跌幅      买入      卖出     昨收     今开  \
0     bj430047  诺思兰德   10.69  -0.28  -2.552   10.69   10.70  10.97  10.88   
1     bj430090  同辉信息    3.46  -0.05  -1.425    3.45    3.46   3.51   3.51   
2     bj430198  微创光电    5.31  -0.02  -0.375    5.29    5.31   5.33   5.29   
3     bj430418  苏轴股份    8.45  -0.04  -0.471    8.44    8.45   8.49   8.50   
4     bj430489  佳先股份    6.10  -0.07  -1.135    6.09    6.10   6.17   6.27   
...        ...   ...     ...    ...     ...     ...     ...    ...    ...   
4837  sz301289  国缆检测   50.26  -0.02  -0.040   50.25   50.26  50.28  50.01   
4838  sz301298  东利机械   24.78   0.07   0.283   24.78   24.80  24.71  24.61   
4839  sz301302  华如科技   64.44  -0.31  -0.479   64.43   64.44  64.75  64.55   
4840  sz301306   C西测   51.53  -3.43  -6.241   51.53   51.54  54.96  54.00   
4841  sz301312   智立方  100.96  11.24  12.528  100.96  100.97  89.72  88.20   

          最高     最低        成交量          成交额  
0      11.21  10.61   592585.

In [5]:
# 0. 准备工作

dbname = "k_house"
#   0.1 创建数据库连接

#  创建数据库 股票分析数据库 的连接 
# conn, curs, engine = common_db.creat_conn_with_pg(dbname=f'{dbname}_for_test')

#  创建数据库 k_house 的连接 
conn2, curs2, engine2 = common_db.creat_conn_with_pg(dbname=dbname)

In [6]:
table_name = 'stock_zh_a_spot_test'

stock_zh_a_spot_df.to_sql(table_name,
          engine2,
          index     = False,
          if_exists = 'replace')    # 写入数据库  还有一个参数是  append，那是附加在表后
# 添加索引
# curs2.execute("""CREATE  INDEX parallel_tool_1      ON  parallel_tool_并行任务分配信息表  (交易日期); 
#                 CREATE  INDEX parallel_tool_2      ON  parallel_tool_并行任务分配信息表  (分组序号);
#                 CREATE  INDEX parallel_tool_3      ON  parallel_tool_并行任务分配信息表  (处理状态);"""
#             )
# conn2.commit()

In [7]:
df = pd.read_sql_query(f"""
    SELECT *
    FROM  {table_name}
  """,
   # params = (date,),
  con = engine2,
)

df[:-5]

,代码,名称,最新价,涨跌额,涨跌幅,买入,卖出,昨收,今开,最高,最低,成交量,成交额
0,bj430047,诺思兰德,10.69,-0.28,-2.552,10.69,10.70,10.97,10.88,11.21,10.61,592585.0,6.417895e+06
1,bj430090,同辉信息,3.46,-0.05,-1.425,3.45,3.46,3.51,3.51,3.51,3.45,952948.0,3.300091e+06
2,bj430198,微创光电,5.31,-0.02,-0.375,5.29,5.31,5.33,5.29,5.32,5.22,95273.0,5.017710e+05
3,bj430418,苏轴股份,8.45,-0.04,-0.471,8.44,8.45,8.49,8.50,8.60,8.40,49840.0,4.247320e+05
4,bj430489,佳先股份,6.10,-0.07,-1.135,6.09,6.10,6.17,6.27,6.27,6.09,171930.0,1.052764e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4832,sz301268,铭利达,57.00,0.65,1.154,56.99,57.00,56.35,55.59,58.70,54.96,3510043.0,1.987015e+08
4833,sz301269,N华大九,75.00,42.31,129.428,75.00,75.01,32.69,69.05,75.50,68.06,50798556.0,3.672527e+09
4834,sz301279,金道科技,28.60,1.32,4.839,28.60,28.61,27.28,27.34,29.30,27.01,5896804.0,1.666081e+08
4835,sz301286,侨源股份,21.86,-0.79,-3.488,21.85,21.86,22.65,22.53,22.76,21.80,3848230.0,8.556259e+07


## 实时行情数据-东财

### 沪深京 A 股
接口: stock_zh_a_spot_em

目标地址: http://quote.eastmoney.com/center/gridlist.html#hs_a_board

描述: 东方财富网-沪深京 A 股-实时行情数据

限量: 单次返回所有沪深京 A 股上市公司的实时行情数据



In [4]:
stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()
print(stock_zh_a_spot_em_df)

        序号      代码    名称    最新价     涨跌幅    涨跌额        成交量           成交额  \
0        1  301269  N华大九  75.00  129.43  42.31   507986.0  3.672527e+09   
1        2  603201   N常润  44.01   44.01  13.45     8412.0  3.686389e+07   
2        3  301018  申菱环境  49.72   20.01   8.29   128865.0  5.854032e+08   
3        4  688170  德龙激光  60.38   19.99  10.06    75180.0  4.322509e+08   
4        5  300848  美瑞新材  23.54   19.98   3.92   184558.0  4.292205e+08   
...    ...     ...   ...    ...     ...    ...        ...           ...   
5059  5060  688607  康众医疗  28.95   -9.98  -3.21    38166.0  1.103905e+08   
5060  5061  603102  百合股份  44.37  -10.00  -4.93    46094.0  2.076029e+08   
5061  5062  002591  恒大高新   9.90  -10.00  -1.10  1151061.0  1.157137e+09   
5062  5063  002824  和胜股份  57.14  -10.00  -6.35    67688.0  3.972610e+08   
5063  5064  603229  奥翔药业  31.49  -10.00  -3.50    53228.0  1.676150e+08   

         振幅     最高  ...     量比    换手率  市盈率-动态   市净率           总市值  \
0     22.76  75.50  ...    NaN

In [8]:
table_name2 = 'stock_zh_a_spot_em_test'

stock_zh_a_spot_em_df.to_sql(table_name2,
          engine2,
          index     = False,
          if_exists = 'replace')    # 写入数据库  还有一个参数是  append，那是附加在表后
# 添加索引
# curs2.execute("""CREATE  INDEX parallel_tool_1      ON  parallel_tool_并行任务分配信息表  (交易日期); 
#                 CREATE  INDEX parallel_tool_2      ON  parallel_tool_并行任务分配信息表  (分组序号);
#                 CREATE  INDEX parallel_tool_3      ON  parallel_tool_并行任务分配信息表  (处理状态);"""
#             )
# conn2.commit()

In [9]:
df2 = pd.read_sql_query(f"""
    SELECT *
    FROM  {table_name2}
  """,
   # params = (date,),
  con = engine2,
)

df2[:-5]

,序号,代码,名称,最新价,涨跌幅,涨跌额,成交量,成交额,振幅,最高,...,量比,换手率,市盈率-动态,市净率,总市值,流通市值,涨速,5分钟涨跌,60日涨跌幅,年初至今涨跌幅
0,1,301269,N华大九,75.00,129.43,42.31,507986.0,3.672527e+09,22.76,75.50,...,NaN,69.17,509.01,9.10,4.072063e+10,5.507667e+09,-0.01,0.27,129.43,129.43
1,2,603201,N常润,44.01,44.01,13.45,8412.0,3.686389e+07,24.02,44.01,...,NaN,4.23,30.03,3.07,3.498502e+09,8.746254e+08,0.00,0.00,44.01,44.01
2,3,301018,申菱环境,49.72,20.01,8.29,128865.0,5.854032e+08,19.89,49.72,...,1.33,15.09,81.38,8.32,1.193330e+10,4.245591e+09,0.00,0.00,132.55,89.05
3,4,688170,德龙激光,60.38,19.99,10.06,75180.0,4.322509e+08,21.36,60.38,...,3.59,35.75,65.81,4.93,6.240877e+09,1.269797e+09,0.00,1.21,122.31,103.44
4,5,300848,美瑞新材,23.54,19.98,3.92,184558.0,4.292205e+08,7.44,23.54,...,13.25,26.61,52.07,5.32,4.708235e+09,1.632664e+09,0.00,0.00,38.15,12.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5054,5055,839725,惠丰钻石,47.99,-9.79,-5.21,15741.0,7.823238e+07,11.24,53.76,...,0.78,14.31,39.10,4.29,2.135555e+09,5.279860e+08,-0.02,-0.72,-28.88,-26.17
5055,5056,000671,阳 光 城,2.29,-9.84,-0.25,4722534.0,1.104449e+09,6.69,2.46,...,1.10,11.65,-1.58,0.57,9.481477e+09,9.279188e+09,0.00,0.00,-8.76,-24.17
5056,5057,603170,宝立食品,30.74,-9.91,-3.38,319506.0,1.051056e+09,15.04,35.84,...,15.11,79.86,73.44,11.94,1.229631e+10,1.229907e+09,-0.16,0.10,205.87,205.87
5057,5058,000056,皇庭国际,5.53,-9.93,-0.61,1205055.0,6.796803e+08,5.70,5.88,...,1.09,13.33,-13.82,1.95,6.495144e+09,4.997620e+09,0.00,0.00,14.73,-3.32


### 沪 A 股

接口: stock_sh_a_spot_em

目标地址: http://quote.eastmoney.com/center/gridlist.html#sh_a_board

描述: 东方财富网-沪 A 股-实时行情数据

限量: 单次返回所有沪 A 股上市公司的实时行情数据

In [10]:
stock_sh_a_spot_em_df = ak.stock_sh_a_spot_em()
print(stock_sh_a_spot_em_df)

        序号      代码    名称    最新价    涨跌幅    涨跌额       成交量           成交额     振幅  \
0        1  603201   N常润  44.01  44.01  13.45    8412.0  3.686389e+07  24.02   
1        2  688170  德龙激光  60.38  19.99  10.06   75180.0  4.322509e+08  21.36   
2        3  688115   思林杰  55.03  14.65   7.03   36476.0  1.906293e+08  15.65   
3        4  688148  芳源股份  23.30  13.66   2.80  189002.0  4.542346e+08   9.71   
4        5  688045   必易微  83.22  12.64   9.34   47699.0  3.808114e+08  15.30   
...    ...     ...   ...    ...    ...    ...       ...           ...    ...   
2196  2197  603170  宝立食品  30.74  -9.91  -3.38  319506.0  1.051056e+09  15.04   
2197  2198  603227  雪峰科技  10.22  -9.96  -1.13  421558.0  4.481957e+08  10.57   
2198  2199  688607  康众医疗  28.95  -9.98  -3.21   38166.0  1.103905e+08  12.03   
2199  2200  603102  百合股份  44.37 -10.00  -4.93   46094.0  2.076029e+08   7.73   
2200  2201  603229  奥翔药业  31.49 -10.00  -3.50   53228.0  1.676150e+08   0.00   

         最高  ...     量比    换手率   市盈率-动态

In [11]:
table_name3 = 'stock_sh_a_spot_em'

stock_sh_a_spot_em_df.to_sql(table_name3,
          engine2,
          index     = False,
          if_exists = 'replace')    # 写入数据库  还有一个参数是  append，那是附加在表后
# 添加索引
# curs2.execute("""CREATE  INDEX parallel_tool_1      ON  parallel_tool_并行任务分配信息表  (交易日期); 
#                 CREATE  INDEX parallel_tool_2      ON  parallel_tool_并行任务分配信息表  (分组序号);
#                 CREATE  INDEX parallel_tool_3      ON  parallel_tool_并行任务分配信息表  (处理状态);"""
#             )
# conn2.commit()

In [12]:
df3 = pd.read_sql_query(f"""
    SELECT *
    FROM  {table_name3}
  """,
   # params = (date,),
  con = engine2,
)

df3[:-5]

,序号,代码,名称,最新价,涨跌幅,涨跌额,成交量,成交额,振幅,最高,...,量比,换手率,市盈率-动态,市净率,总市值,流通市值,涨速,5分钟涨跌,60日涨跌幅,年初至今涨跌幅
0,1,603201,N常润,44.01,44.01,13.45,8412.0,36863886.0,24.02,44.01,...,NaN,4.23,30.03,3.07,3.498502e+09,8.746254e+08,0.00,0.00,44.01,44.01
1,2,688170,德龙激光,60.38,19.99,10.06,75180.0,432250911.0,21.36,60.38,...,3.59,35.75,65.81,4.93,6.240877e+09,1.269797e+09,0.00,1.21,122.31,103.44
2,3,688115,思林杰,55.03,14.65,7.03,36476.0,190629274.0,15.65,55.59,...,3.02,23.82,57.87,2.84,3.668850e+09,8.427627e+08,0.02,0.29,55.36,-15.60
3,4,688148,芳源股份,23.30,13.66,2.80,189002.0,454234557.0,9.71,24.60,...,6.13,24.10,-1312.66,9.10,1.192303e+10,1.827165e+09,-0.43,-0.89,57.75,-33.26
4,5,688045,必易微,83.22,12.64,9.34,47699.0,380811377.0,15.30,84.58,...,1.66,32.57,49.24,4.27,5.746253e+09,1.218896e+09,0.19,-0.87,50.90,50.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2191,2192,688366,昊海生科,90.24,-6.97,-6.76,18310.0,169314746.0,8.42,97.50,...,1.98,3.23,64.46,2.80,1.571349e+10,5.119225e+09,-0.03,-0.14,16.48,-28.32
2192,2193,688130,N晶华微,58.15,-7.67,-4.83,77967.0,458574931.0,9.99,62.78,...,NaN,52.55,40.43,2.96,3.870464e+09,8.627021e+08,0.17,0.26,-7.67,-7.67
2193,2194,603261,立航科技,46.76,-7.95,-4.04,41376.0,196073331.0,5.87,49.48,...,1.85,21.49,189.79,4.24,3.598735e+09,9.001300e+08,0.00,0.00,9.46,140.78
2194,2195,688206,概伦电子,31.00,-8.26,-2.79,46386.0,147940240.0,7.99,33.70,...,2.24,11.18,715.38,6.38,1.344794e+10,1.286404e+09,0.00,-0.29,48.47,-15.46


### 深 A 股

接口: stock_sz_a_spot_em

目标地址: http://quote.eastmoney.com/center/gridlist.html#sz_a_board

描述: 东方财富网-深 A 股-实时行情数据

限量: 单次返回所有深 A 股上市公司的实时行情数据

In [13]:
stock_sz_a_spot_em_df = ak.stock_sz_a_spot_em()
print(stock_sz_a_spot_em_df)

        序号      代码     名称    最新价     涨跌幅    涨跌额        成交量           成交额  \
0        1  301269   N华大九  75.00  129.43  42.31   507986.0  3.672527e+09   
1        2  301018   申菱环境  49.72   20.01   8.29   128865.0  5.854032e+08   
2        3  300848   美瑞新材  23.54   19.98   3.92   184558.0  4.292205e+08   
3        4  300062   中能电气   8.83   19.97   1.47  1020359.0  8.778986e+08   
4        5  300044   赛为智能   4.21   19.94   0.70  1095230.0  4.385944e+08   
...    ...     ...    ...    ...     ...    ...        ...           ...   
2754  2755  001336   楚环科技  39.81   -9.54  -4.20   131320.0  5.440592e+08   
2755  2756  000671  阳 光 城   2.29   -9.84  -0.25  4722534.0  1.104449e+09   
2756  2757  000056   皇庭国际   5.53   -9.93  -0.61  1205055.0  6.796803e+08   
2757  2758  002591   恒大高新   9.90  -10.00  -1.10  1151061.0  1.157137e+09   
2758  2759  002824   和胜股份  57.14  -10.00  -6.35    67688.0  3.972610e+08   

         振幅     最高  ...     量比    换手率  市盈率-动态   市净率           总市值  \
0     22.76  75.50

In [14]:
table_name4 = 'stock_sz_a_spot_em'

stock_sz_a_spot_em_df.to_sql(table_name4,
          engine2,
          index     = False,
          if_exists = 'replace')    # 写入数据库  还有一个参数是  append，那是附加在表后
# 添加索引
# curs2.execute("""CREATE  INDEX parallel_tool_1      ON  parallel_tool_并行任务分配信息表  (交易日期); 
#                 CREATE  INDEX parallel_tool_2      ON  parallel_tool_并行任务分配信息表  (分组序号);
#                 CREATE  INDEX parallel_tool_3      ON  parallel_tool_并行任务分配信息表  (处理状态);"""
#             )
# conn2.commit()

In [15]:
df4 = pd.read_sql_query(f"""
    SELECT *
    FROM  {table_name4}
  """,
   # params = (date,),
  con = engine2,
)

df4[:-5]

,序号,代码,名称,最新价,涨跌幅,涨跌额,成交量,成交额,振幅,最高,...,量比,换手率,市盈率-动态,市净率,总市值,流通市值,涨速,5分钟涨跌,60日涨跌幅,年初至今涨跌幅
0,1,301269,N华大九,75.00,129.43,42.31,507986.0,3.672527e+09,22.76,75.50,...,NaN,69.17,509.01,9.10,4.072063e+10,5.507667e+09,-0.01,0.27,129.43,129.43
1,2,301018,申菱环境,49.72,20.01,8.29,128865.0,5.854032e+08,19.89,49.72,...,1.33,15.09,81.38,8.32,1.193330e+10,4.245591e+09,0.00,0.00,132.55,89.05
2,3,300848,美瑞新材,23.54,19.98,3.92,184558.0,4.292205e+08,7.44,23.54,...,13.25,26.61,52.07,5.32,4.708235e+09,1.632664e+09,0.00,0.00,38.15,12.20
3,4,300062,中能电气,8.83,19.97,1.47,1020359.0,8.778986e+08,12.23,8.83,...,5.44,32.49,116.04,4.06,4.923408e+09,2.772937e+09,0.00,0.00,38.18,25.60
4,5,300044,赛为智能,4.21,19.94,0.70,1095230.0,4.385944e+08,21.08,4.21,...,4.68,16.37,186.86,3.12,3.242379e+09,2.817328e+09,0.00,0.00,57.68,26.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2749,2750,000544,中原环保,7.30,-8.64,-0.69,755571.0,5.764380e+08,12.14,8.16,...,0.88,7.75,13.82,1.11,7.115197e+09,7.115197e+09,0.00,-0.95,33.46,18.70
2750,2751,002395,双象股份,9.94,-8.81,-0.96,239469.0,2.434403e+08,6.88,10.56,...,1.93,8.93,-111.99,2.97,2.665997e+09,2.665997e+09,0.30,0.20,31.83,-4.15
2751,2752,000595,宝塔实业,6.13,-8.92,-0.60,1411119.0,9.178411e+08,11.59,6.90,...,1.22,12.40,-132.48,9.42,6.979964e+09,6.976210e+09,0.00,-1.45,105.02,92.16
2752,2753,300066,三川智慧,6.25,-9.16,-0.63,723038.0,4.661977e+08,9.88,6.89,...,2.79,7.21,47.74,3.17,6.500208e+09,6.271426e+09,0.16,0.00,41.72,25.25
